In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# prompt:

# Check if the device is CUDA
if device == "cuda":
    # Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    # Print the number of available CUDA devices
    print(f"Number of available CUDA devices: {num_cuda_devices}")
    # Get the name of the current CUDA device
    current_cuda_device = torch.cuda.current_device()
    # Print the name of the current CUDA device
    print(f"Current CUDA device: {torch.cuda.get_device_name(current_cuda_device)}")
else:
    # Print a message indicating that CUDA is not available
    print("CUDA is not available.")


NameError: name 'device' is not defined

In [3]:

def dice_score_3d(predicted, target, epsilon=1e-8):
    """
    Compute the Dice score (F1 score) for 3D binary segmentation.

    Args:
    - predicted (torch.Tensor): Predicted binary segmentation mask (0 or 1).
    - target (torch.Tensor): Ground truth binary segmentation mask (0 or 1).
    - epsilon (float, optional): Small value added to the denominator to avoid division by zero.

    Returns:
    - dice_score (float): Dice score (F1 score) between predicted and target masks.
    """

    # Flatten tensors to vectors
    predicted_flat = predicted.view(-1)
    target_flat = target.view(-1)

    # Calculate true positive (TP), false positive (FP), and false negative (FN)
    intersection = torch.sum(predicted_flat * target_flat)
    total_predicted = torch.sum(predicted_flat)
    total_target = torch.sum(target_flat)
    dice_score = (2.0 * intersection + epsilon) / (total_predicted + total_target + epsilon)

    return dice_score.item()


In [2]:
torch.manual_seed(42)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class medt_net_3d(nn.Module):
    def __init__(self, block, block_2, layers, num_classes=2, zero_init_residual=True,
                 groups=8, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None, s=0.125, img_size=256, imgchan=1):
        super(medt_net_3d, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm3d
        self._norm_layer = norm_layer

        self.inplanes = int(64 * s)
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv3d(imgchan, self.inplanes, kernel_size=7, stride=1, padding=3, bias=False)
        self.conv2 = nn.Conv3d(self.inplanes, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv3d(128, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.bn2 = norm_layer(128)
        self.bn3 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)


        self.layer1 = self._make_layer(block, int(128 * s), layers[0], kernel_size=(img_size // 2))
        self.layer2 = self._make_layer(block, int(256 * s), layers[1], stride=2, kernel_size=(img_size // 2),
                                         dilate=replace_stride_with_dilation[0])

        self.layer3 = self._make_layer(block, int(512 * s), layers[2], stride=2, kernel_size=(img_size//4),
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, int(1024 * s), layers[3], stride=2, kernel_size=(img_size//8),
                                       dilate=replace_stride_with_dilation[2])

        # Decoder
        self.decoder1 = nn.Conv3d(int(1024 *2*s)      ,        int(1024*2*s), kernel_size=3, stride=2, padding=1)
        self.decoder2 = nn.Conv3d(int(1024  *2*s)     , int(1024*s), kernel_size=3, stride=1, padding=1)
        self.decoder3 = nn.Conv3d(int(1024*s),  int(512*s), kernel_size=3, stride=1, padding=1)
        self.decoder4 = nn.Conv3d(int(512*s) ,  int(256*s), kernel_size=3, stride=1, padding=1)
        self.decoder5 = nn.Conv3d(int(256*s) , int(128*s) , kernel_size=3, stride=1, padding=1)
        self.adjust   = nn.Conv3d(int(128*s) , num_classes, kernel_size=1, stride=1, padding=0)
        self.soft     = nn.Softmax(dim=1)


        self.conv1_p = nn.Conv3d(imgchan, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.conv2_p = nn.Conv3d(self.inplanes,128, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv3_p = nn.Conv3d(128, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        # self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1_p = norm_layer(self.inplanes)
        self.bn2_p = norm_layer(128)
        self.bn3_p = norm_layer(self.inplanes)

        self.relu_p = nn.ReLU(inplace=True)

        img_size_p = img_size // 2

        self.layer1_p = self._make_layer(block_2, int(128 * s), layers[0], kernel_size= (img_size_p//2))
        self.layer2_p = self._make_layer(block_2, int(256 * s), layers[1], stride=2, kernel_size=(img_size_p//2),
                                       dilate=replace_stride_with_dilation[0])
        self.layer3_p = self._make_layer(block_2, int(512 * s), layers[2], stride=2, kernel_size=(img_size_p//4),
                                       dilate=replace_stride_with_dilation[1])
        self.layer4_p = self._make_layer(block_2, int(1024 * s), layers[3], stride=2, kernel_size=(img_size_p//8),
                                       dilate=replace_stride_with_dilation[2])

        # Decoder
        self.decoder1_p = nn.Conv3d(int(1024 *2*s)      ,        int(1024*2*s), kernel_size=3, stride=2, padding=1)
        self.decoder2_p = nn.Conv3d(int(1024  *2*s)     , int(1024*s), kernel_size=3, stride=1, padding=1)
        self.decoder3_p = nn.Conv3d(int(1024*s),  int(512*s), kernel_size=3, stride=1, padding=1)
        self.decoder4_p = nn.Conv3d(int(512*s) ,  int(256*s), kernel_size=3, stride=1, padding=1)
        self.decoder5_p = nn.Conv3d(int(256*s) , int(128*s) , kernel_size=3, stride=1, padding=1)

        self.decoderf = nn.Conv3d(int(128*s) , int(128*s) , kernel_size=3, stride=1, padding=1)
        self.adjust_p   = nn.Conv3d(int(128*s) , num_classes, kernel_size=1, stride=1, padding=0)
        self.soft_p     = nn.Softmax(dim=1)

    def _make_layer(self, block, planes, blocks, kernel_size=56, stride=1, dilate=False):
        norm_layer = nn.BatchNorm3d
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1_3d(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, groups=self.groups,
                            base_width=self.base_width, dilation=previous_dilation,
                            norm_layer=norm_layer, kernel_size=kernel_size))
        self.inplanes = planes * block.expansion
        if stride != 1:
            kernel_size = kernel_size // 2

        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer, kernel_size=kernel_size))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        xin = x.clone()
        print(x.shape)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        print(x.shape)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        print(x.shape)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.max_pool3d(x, (2, 2, 2))
        x = self.relu(x)
        print(x.shape)
        # N, C, H, W, D = x.shape
        # x = x.view(N,C,H*W,D)

        # x = self.maxpool(x)
        # pdb.set_trace()
        x1 = self.layer1(x)
        # print(x1.shape)
        x2 = self.layer2(x1)
        # print(x2.shape)
        x3 = self.layer3(x2)
        # # print(x3.shape)
        x4 = self.layer4(x3)
        # # print(x4.shape)
        x = F.relu(F.interpolate(self.decoder1(x4), scale_factor=(2,2,2), mode ='trilinear'))
        x = torch.add(x, x4)
        x = F.relu(F.interpolate(self.decoder2(x4) , scale_factor=(2,2,2), mode ='trilinear'))
        x = torch.add(x, x3)
        x = F.relu(F.interpolate(self.decoder3(x3) , scale_factor=(2,2,2), mode ='trilinear'))
        x = torch.add(x, x2)
        x = F.relu(F.interpolate(self.decoder4(x2) , scale_factor=(2,2,2), mode ='trilinear'))
        x = torch.add(x, x1)
        x = F.relu(F.interpolate(self.decoder5(x) , scale_factor=(2,2,2), mode ='trilinear'))
        print(x.shape)
        # N, C, _, _ = x.shape
        # x = x.view(N,C,256,256,256)
        # print(x.shape)

        # end of full image training

        # y_out = torch.ones((1,2,128,128))
        x_loc = x.clone()
        # x = F.relu(F.interpolate(self.decoder5(x) , scale_factor=(2,2), mode ='bilinear'))
        #start
        for i in range(0,2):
            for j in range(0,2):
              for k in range(0,2):

                x_p = xin[:,128*i:128*(i+1),128*j:128*(j+1),128*k:128*(k+1)]
                # begin patch wise
                x_p = self.conv1_p(x_p)
                x_p = self.bn1_p(x_p)
                # x = F.max_pool2d(x,2,2)
                x_p = self.relu(x_p)

                x_p = self.conv2_p(x_p)
                x_p = self.bn2_p(x_p)
                # x = F.max_pool2d(x,2,2)
                x_p = self.relu(x_p)
                x_p = self.conv3_p(x_p)
                x_p = self.bn3_p(x_p)
                # x = F.max_pool2d(x,2,2)
                x_p = self.relu(x_p)
                N, C, H, W, D = x_p.shape
                x_p = x_p.view(N,C,H,W*D)

                # x = self.maxpool(x)
                # pdb.set_trace()
                x1_p = self.layer1_p(x_p)
                # print(x1.shape)
                x2_p = self.layer2_p(x1_p)
                # print(x2.shape)
                x3_p = self.layer3_p(x2_p)
                # # print(x3.shape)
                x4_p = self.layer4_p(x3_p)

                x_p = F.relu(F.interpolate(self.decoder1_p(x4_p), scale_factor=(2,2,2), mode ='trilinear'))
                x_p = torch.add(x_p, x4_p)
                x_p = F.relu(F.interpolate(self.decoder2_p(x_p) , scale_factor=(2,2,2), mode ='trilinear'))
                x_p = torch.add(x_p, x3_p)
                x_p = F.relu(F.interpolate(self.decoder3_p(x_p) , scale_factor=(2,2,2), mode ='trilinear'))
                x_p = torch.add(x_p, x2_p)
                x_p = F.relu(F.interpolate(self.decoder4_p(x_p) , scale_factor=(2,2,2), mode ='trilinear'))
                x_p = torch.add(x_p, x1_p)
                x_p = F.relu(F.interpolate(self.decoder5_p(x_p) , scale_factor=(2,2,2), mode ='trilinear'))
                N, C, _, _ = x_p.shape
                x_p = x_p.view(N,C,128,128,128)

                x_loc[:,128*i:128*(i+1),128*j:128*(j+1),128*k:128*(k+1)] = x_p

        x = torch.add(x,x_loc)
        x = F.relu(self.decoderf(x))

        x = self.adjust(F.relu(x))

        # pdb.set_trace()
        return x

    def forward(self, x):
        return self._forward_impl(x)




# Define the 3D block used in the network (replace with your specific 3D block)

def conv1x1_3d(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class qkv_transform_3d(nn.Conv2d):
    """Conv1d for qkv_transform"""

class AxialAttention_dynamic_3d(nn.Module):
    def __init__(self, in_planes, out_planes, groups=8, kernel_size=56,
                 stride=1, bias=False, width=0):
        assert (in_planes % groups == 0) and (out_planes % groups == 0)
        super(AxialAttention_dynamic_3d, self).__init__()
        self.in_planes = in_planes
        self.out_planes = out_planes
        self.groups = groups
        self.group_planes = out_planes // groups
        self.kernel_size = kernel_size
        self.stride = stride
        self.bias = bias
        self.width = width

        # Multi-head self attention
        self.qkv_transform = qkv_transform_3d(in_planes, out_planes * 2, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn_qkv = nn.BatchNorm2d(out_planes * 2)
        self.bn_similarity = nn.BatchNorm3d(groups * 3)
        self.bn_output = nn.BatchNorm2d(out_planes * 2)

        # Priority on encoding

        ## Initial values

        self.f_qr = nn.Parameter(torch.tensor(0.1), requires_grad=False)
        self.f_kr = nn.Parameter(torch.tensor(0.1), requires_grad=False)
        self.f_sve = nn.Parameter(torch.tensor(0.1), requires_grad=False)
        self.f_sv = nn.Parameter(torch.tensor(1.0), requires_grad=False)

        # Position embedding
        self.relative = nn.Parameter(torch.randn(self.group_planes * 2, kernel_size * 2 - 1, kernel_size * 2 - 1), requires_grad=True)
        query_index = torch.arange(kernel_size).unsqueeze(0)
        key_index = torch.arange(kernel_size).unsqueeze(1)
        relative_index = key_index - query_index + kernel_size - 1
        self.register_buffer('flatten_index', relative_index.view(-1))
        if stride > 1:
            self.pooling = nn.AvgPool3d(stride, stride=stride)

        self.reset_parameters()
        # self.print_para()

    def forward(self, x):
        if self.width == 0:
            x = x.permute(0, 2, 1, 3, 4)
        elif self.width == 1:
            x = x.permute(0, 3, 1, 2, 4)  # N, D, C, H, W
        else:
            x = x.permute(0, 4, 1, 2, 3)
        N, D, C, H, W = x.shape
        print("The shape of x before the layer", x.shape)
        x = x.contiguous().view(N * D, C, H, W)

        # Transformations
        qkv = self.bn_qkv(self.qkv_transform(x))
        q, k, v = torch.split(qkv.reshape(N * D, self.groups, self.group_planes * 2, H, W),
                              [self.group_planes // 2, self.group_planes // 2, self.group_planes], dim=2)

        # Calculate position embedding
        all_embeddings = torch.index_select(self.relative, 1, self.flatten_index).view(self.group_planes * 2, self.kernel_size, self.kernel_size, self.kernel_size)
        q_embedding, k_embedding, v_embedding = torch.split(all_embeddings, [self.group_planes // 2, self.group_planes // 2, self.group_planes], dim=0)
        qr = torch.einsum('bgcik,cijk->bgijk', q, q_embedding)
        kr = torch.einsum('bgcik,cijk->bgijk', k, k_embedding).transpose(2, 3)
        qk = torch.einsum('bgcik, bgcjk->bgijk', q, k)

        # multiply by factors
        qr = torch.mul(qr, self.f_qr)
        kr = torch.mul(kr, self.f_kr)

        stacked_similarity = torch.cat([qk, qr, kr], dim=1)
        stacked_similarity = self.bn_similarity(stacked_similarity).view(N * D, 3, self.groups, H, W, H).sum(dim=1)
        # stacked_similarity = self.bn_qr(qr) + self.bn_kr(kr) + self.bn_qk(qk)
        # (N, groups, H, W, D)
        similarity = F.softmax(stacked_similarity, dim=4)
        sv = torch.einsum('bgijk,bgcijk->bgcijk', similarity, v)
        sve = torch.einsum('bgijk,cijk->bgcik', similarity, v_embedding)

        # multiply by factors
        sv = torch.mul(sv, self.f_sv)
        sve = torch.mul(sve, self.f_sve)

        stacked_output = torch.cat([sv, sve], dim=-1).view(N * D, self.out_planes * 2, H, W)
        output = self.bn_output(stacked_output).view(N, D, self.out_planes, 2, H, W).sum(dim=-3)

        if self.width == 0:
            output = output.permute(0, 2, 1, 3, 4)
        elif  self.width ==1:
            output = output.permute(0, 2, 3, 1, 4)
        else:
            output = output.permute(0, 2, 3, 4, 1)

        if self.stride > 1:
            output = self.pooling(output)

        return output

    def reset_parameters(self):
        self.qkv_transform.weight.data.normal_(0, math.sqrt(1. / self.in_planes))
        # nn.init.uniform_(self.relative, -0.1, 0.1)
        nn.init.normal_(self.relative, 0., math.sqrt(1. / self.group_planes))



class AxialBlock_dynamic_3d(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None, kernel_size=56):
        super(AxialBlock_dynamic_3d, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm3d
        width = int(planes * (base_width / 64.))
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv_down = conv1x1_3d(inplanes, width)
        self.bn1 = norm_layer(width)
        self.hight_block = AxialAttention_dynamic_3d(width, width, groups=groups, kernel_size=kernel_size)
        self.width_block = AxialAttention_dynamic_3d(width, width, groups=groups, kernel_size=kernel_size, stride=stride, width=1)
        self.depth_block = AxialAttention_dynamic_3d(width, width, groups=groups, kernel_size=kernel_size, stride=stride, width=2)
        self.conv_up = conv1x1_3d(width, planes * self.expansion)
        self.bn2 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        print("shape of x before convdown", x.shape)
        out = self.conv_down(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.hight_block(out)
        out = self.width_block(out)
        out = self.depth_block(out)
        out = self.relu(out)

        out = self.conv_up(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
def MedT3d(pretrained=False, **kwargs):
  # Instantiate the modified 3D network
  model_3d = medt_net_3d(AxialBlock_dynamic_3d, AxialBlock_dynamic_3d, layers=[3, 4, 6, 3], num_classes=2)
  return model_3d

In [6]:
import os
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

class NiftiDataset(Dataset):
    def __init__(self, data_dir, label_dir, train=True):
        self.data_dir = data_dir
        self.label_dir = label_dir
        self.data_files = [f for f in os.listdir(data_dir) if f.endswith('.nii.gz')]
        self.label_files = [f for f in os.listdir(label_dir) if f.endswith('.nii.gz')]
        self.train = train

        if train:
            # Use 80% of the data for training
            self.data_files, _ = train_test_split(self.data_files, test_size=0.2, random_state=42)

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        data_path = os.path.join(self.data_dir, self.data_files[idx])
        label_path = os.path.join(self.label_dir, self.label_files[idx])

        nifti_data = nib.load(data_path).get_fdata()
        tensor_data = torch.from_numpy(nifti_data).float()

        nifti_label = nib.load(label_path).get_fdata()
        tensor_label = torch.from_numpy(nifti_label).float()

        return tensor_data, tensor_label

# Read the file paths and return a dataset class.
def readData(data_path='/content/data/Datapoint',
             label_path='/content/data/Annotation'):
    # Use 80% of the data for training, 20% for testing
    train_dataset = NiftiDataset(data_path, label_path, train=True)
    test_dataset = NiftiDataset(data_path, label_path, train=False)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    return train_loader, test_loader


In [7]:
import os

import torch
import argparse


train_data, test_data = readData()


model = MedT3d(img_size = 256, imgchan = 1)

criterion = dice_score_3d
optimizer = torch.optim.Adam(list(model.parameters()), lr=0.001,
                             weight_decay=1e-5)
padding_size = (0, 0, 64, 64)
for batch_idx, (X_batch, y_batch, *rest) in enumerate(train_data):

  print(X_batch.shape)
  print(y_batch.shape)



save_dir = "/content/data/output"

for epoch in range(40):

    epoch_running_loss = 0

    for batch_idx, (X_batch, y_batch, *rest) in enumerate(train_data):
        padded_X_batch = F.pad(X_batch, padding_size, value=-1)
        padded_y_batch = F.pad(y_batch, padding_size, value=0)

        X_batch = padded_X_batch.to(device = 'cpu').permute(0, 4, 1, 2, 3)
        y_batch = padded_y_batch.to(device = 'cpu').permute(0, 4, 1, 2, 3)

        print(padded_X_batch.shape)
        print(padded_y_batch.shape)

        # ===================forward=====================


        output = model(X_batch)

        tmp2 = y_batch[0].detach().cpu().numpy()
        tmp = output.detach().cpu().numpy()
        tmp[tmp>=0.5] = 1
        tmp[tmp<0.5] = 0
        tmp2[tmp2>0] = 1
        tmp2[tmp2<=0] = 0
        tmp2 = tmp2.astype(int)
        tmp = tmp.astype(int)

        yHaT = tmp
        yval = tmp2



        loss = criterion(output, y_batch[0])

        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_running_loss += loss.item()

    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch, 40, epoch_running_loss/(batch_idx+1)))


    if epoch == 10:
        for param in model.parameters():
            param.requires_grad =True
    if (epoch % 4) ==0:

        # for batch_idx, (X_batch, y_batch, *rest) in enumerate(valloader):
        #     # print(batch_idx)
        #     if isinstance(rest[0][0], str):
        #                 image_filename = rest[0][0]
        #     else:
        #                 image_filename = '%s.png' % str(batch_idx + 1).zfill(3)

        #     X_batch = Variable(X_batch.to(device='cpu'))
        #     y_batch = Variable(y_batch.to(device='cpu'))
        #     # start = timeit.default_timer()
        #     y_out = model(X_batch)
        #     # stop = timeit.default_timer()
        #     # print('Time: ', stop - start)
        #     tmp2 = y_batch.detach().cpu().numpy()
        #     tmp = y_out.detach().cpu().numpy()
        #     tmp[tmp>=0.5] = 1
        #     tmp[tmp<0.5] = 0
        #     tmp2[tmp2>0] = 1
        #     tmp2[tmp2<=0] = 0
        #     tmp2 = tmp2.astype(int)
        #     tmp = tmp.astype(int)

        #     # print(np.unique(tmp2))
        #     yHaT = tmp
        #     yval = tmp2

        #     epsilon = 1e-20

        #     del X_batch, y_batch,tmp,tmp2, y_out


        #     yHaT[yHaT==1] =255
        #     yval[yval==1] =255
        #     fulldir = direc+"/{}/".format(epoch)
        #     # print(fulldir+image_filename)
        #     if not os.path.isdir(fulldir):

        #         os.makedirs(fulldir)

        #     cv2.imwrite(fulldir+image_filename, yHaT[0,1,:,:])
        #     # cv2.imwrite(fulldir+'/gt_{}.png'.format(count), yval[0,:,:])
        # fulldir = direc+"/{}/".format(epoch)
        # torch.save(model.state_dict(), fulldir+args.modelname+".pth")
        # torch.save(model.state_dict(), direc+"final_model.pth")

        # Save model checkpoint
        fulldir = save_dir + "/{}/".format(epoch)
        if not os.path.isdir(fulldir):
            os.makedirs(fulldir)

        torch.save(model.state_dict(), fulldir + "MedT3d" + ".pth")

torch.save(model.state_dict(), save_dir + "final_model.pth")




torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 128, 1])
torch.Size([1, 256, 256, 256, 1])
torch.Size([1, 256, 256, 256, 1])
torch.Size([1, 1, 256, 256, 256])
torch.Size([1, 8, 256, 256, 256])
torch.Size([1, 128, 256, 256, 256])
torch.Size([1, 8, 128, 128, 128])
shape of x before convdown torch.Size([1, 8, 128, 128, 128])
The shape of x before the layer torch.Size([1, 128, 16, 128, 128])


RuntimeError: shape '[4, 128, 128, 128]' is invalid for input of size 16711680